In [9]:
# import required libraries
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor

In [10]:
# define function to load data
def load_data(filepath):
    # setting column names of the dataframe
    headers = [
        "X_robot",
        "Y_robot",
        "Orientation_robot",
        "Collision",
        "X_candle1",
        "Y_candle1",
        "X_candle2",
        "Y_candle2",
        "X_candle3",
        "Y_candle3",
        "X_candle4",
        "Y_candle4",
        "X_speed",
        "Y_speed",
    ]

    # load data from file
    df = pd.read_csv(filepath, names=headers)

    return df

In [11]:
# function to calculate Euclidean distance between two points
def euclidean_distance(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5


# function that performs feature engineering on the dataset
def feature_engineering(robots_df):
    # calculating the distance between the robot and each candle
    robots_df["distance_candle1"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle1"],
        robots_df["Y_candle1"],
    )
    robots_df["distance_candle2"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle2"],
        robots_df["Y_candle2"],
    )
    robots_df["distance_candle3"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle3"],
        robots_df["Y_candle3"],
    )
    robots_df["distance_candle4"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle4"],
        robots_df["Y_candle4"],
    )

    # calculating the distance between the robot and each candle in each axis
    robots_df["distance_candle1_x"] = robots_df["X_robot"] - robots_df["X_candle1"]
    robots_df["distance_candle1_y"] = robots_df["Y_robot"] - robots_df["Y_candle1"]
    robots_df["distance_candle2_x"] = robots_df["X_robot"] - robots_df["X_candle2"]
    robots_df["distance_candle2_y"] = robots_df["Y_robot"] - robots_df["Y_candle2"]
    robots_df["distance_candle3_x"] = robots_df["X_robot"] - robots_df["X_candle3"]
    robots_df["distance_candle3_y"] = robots_df["Y_robot"] - robots_df["Y_candle3"]
    robots_df["distance_candle4_x"] = robots_df["X_robot"] - robots_df["X_candle4"]
    robots_df["distance_candle4_y"] = robots_df["Y_robot"] - robots_df["Y_candle4"]

    return robots_df

In [12]:
# function to develop and save the model
def develop(X, y):
    # split the data into training, validation and test sets
    X_train_validate, X_test, y_train_validate, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    X_train, X_validate, y_train, y_validate = train_test_split(
        X_train_validate, y_train_validate, test_size=0.125, random_state=42
    )

    # scale the data
    scaler = StandardScaler().fit(X_train)

    X_train_scaled = scaler.transform(X_train)
    X_validate_scaled = scaler.transform(X_validate)
    X_test_scaled = scaler.transform(X_test)

    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=X_validate.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    # apply PCA to reduce the dimensionality of the data
    pca = PCA(n_components=0.95).fit(X_train_scaled)
    x_train_pca = pca.transform(X_train_scaled)
    x_validate_pca = pca.transform(X_validate_scaled)
    x_test_pca = pca.transform(X_test_scaled)

    # train the model
    hyper_params = {
        # "hidden_layer_sizes": (100, 100),
        # "activation": "relu",
        # "solver": "adam",
        # "max_iter": 1000,
        # "random_state": 33,
        # "batch_size": 32,
        # "learning_rate_init": "0.001",
        "verbose": False,
    }
    model = MLPRegressor(**hyper_params).fit(x_train_pca, y_train)

    # evaluate the model
    train_score = model.score(x_train_pca, y_train)
    validate_score = model.score(x_validate_pca, y_validate)
    test_score = model.score(x_test_pca, y_test)

    # model props
    model_props = {
        "id": id,
        "hidden_layers": model.n_layers_ - 2,
        "architecture": model.hidden_layer_sizes,
        "activation": model.activation,
        "batch_size": model.batch_size,
        "solver": model.solver,
        "learning_rate_init": model.learning_rate_init,
        "learning_rate": model.learning_rate,
        "iterations": model.n_iter_,
        "train_score": train_score,
        "val_score": validate_score,
        "test_score": test_score,
    }

    # print the model properties
    print(model_props)

    # save the scaler
    with open("scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)

    # save the pca
    with open("pca.pkl", "wb") as f:
        pickle.dump(pca, f)

    # save the model
    with open("model.pkl", "wb") as f:
        pickle.dump(model, f)

In [13]:
# function to evaluate the model
def evaluate(X, y):
    scaler = pickle.load(open("scaler.pkl", "rb"))
    model = pickle.load(open("model.pkl", "rb"))
    pca = pickle.load(open("pca.pkl", "rb"))

    # scale the data
    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

    # apply PCA to reduce the dimensionality of the data
    x_pca = pca.transform(X_scaled)

    # evaluate the model
    test_score = model.score(x_pca, y)

    # model props
    model_props = {
        "id": id,
        "hidden_layers": model.n_layers_ - 2,
        "architecture": model.hidden_layer_sizes,
        "activation": model.activation,
        "batch_size": model.batch_size,
        "solver": model.solver,
        "learning_rate_init": model.learning_rate_init,
        "learning_rate": model.learning_rate,
        "iterations": model.n_iter_,
        "test_score": test_score,
    }

    # print the model properties
    print(model_props)

In [14]:
# the main function that runs the program
def main(mode="develop", file_path="./data/RobotData.csv"):

    # Load data
    df = load_data(file_path)

    # data augmentation needs to be done here, if required

    # Perform feature engineering
    df_fe = feature_engineering(df)

    # print head of the dataframe
    # print(df_fe.head())

    # selected features
    selected_features = [
        "distance_candle1",
        "distance_candle2",
        "distance_candle3",
        "distance_candle4",
        "distance_candle1_x",
        "distance_candle1_y",
        "distance_candle2_x",
        "distance_candle2_y",
        "distance_candle3_x",
        "distance_candle3_y",
        "distance_candle4_x",
        "distance_candle4_y",
    ]

    # target variables
    target = ["X_speed", "Y_speed"]

    # get feature and target vectors
    X = df_fe[selected_features]
    y = df_fe[target]

    # scale the data
    if mode == "develop":
        develop(X, y)
    else:
        evaluate(X, y)

In [15]:
# run the main function
main("develop", "./data/RobotData.csv")

{'id': <built-in function id>, 'hidden_layers': 1, 'architecture': (100,), 'activation': 'relu', 'batch_size': 'auto', 'solver': 'adam', 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'iterations': 17, 'train_score': 0.09927035729657985, 'val_score': 0.05454017370733505, 'test_score': 0.04584405168090577}
